# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,quatre cocos,-20.2078,57.7625,21.72,63,54,9.66,MU,1724771569
1,1,pervomayskoye,43.9190,46.7102,28.80,30,90,6.61,RU,1724771570
2,2,wailua homesteads,22.0669,-159.3780,23.75,84,75,7.20,US,1724771572
3,3,palmer,42.1584,-72.3287,26.29,57,0,1.52,US,1724771573
4,4,edinburgh of the seven seas,-37.0676,-12.3116,10.39,68,100,8.17,SH,1724771574


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 21)] # Narrow down to temperature between 18 to 26C
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Humidity"] < 80) & (ideal_weather_df["Humidity"] > 30)] # Narrow down to humidity between 30 to 70%
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10] # Narrow down to speed less than 10m/s.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0] # Narrow down to cloudiness = 0%

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,palmer,42.1584,-72.3287,26.29,57,0,1.52,US,1724771573
16,16,crane,31.3974,-102.3501,25.33,60,0,2.68,US,1724771587
81,81,bethel,41.3712,-73.4140,27.17,67,0,0.89,US,1724771667
96,96,tolanaro,-25.0319,46.9987,22.07,79,0,8.51,MG,1724771684
120,120,majdal shams,33.2684,35.7694,22.37,41,0,4.05,IL,1724771712
166,166,tsiombe,-25.3000,45.4833,22.84,62,0,3.15,MG,1724771768
184,184,lata,40.1629,-8.3327,29.72,36,0,4.60,PT,1724771789
194,194,bilajer,40.5681,50.0378,27.90,54,0,5.14,AZ,1724771801
205,205,port alfred,-33.5906,26.8910,21.77,60,0,0.89,ZA,1724771814
248,248,greenwood,34.1668,-82.1165,29.34,59,0,2.57,US,1724771864


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,palmer,42.1584,-72.3287,26.29,57,0,1.52,US,1724771573,
16,16,crane,31.3974,-102.3501,25.33,60,0,2.68,US,1724771587,
81,81,bethel,41.3712,-73.4140,27.17,67,0,0.89,US,1724771667,
96,96,tolanaro,-25.0319,46.9987,22.07,79,0,8.51,MG,1724771684,
120,120,majdal shams,33.2684,35.7694,22.37,41,0,4.05,IL,1724771712,
166,166,tsiombe,-25.3000,45.4833,22.84,62,0,3.15,MG,1724771768,
184,184,lata,40.1629,-8.3327,29.72,36,0,4.60,PT,1724771789,
194,194,bilajer,40.5681,50.0378,27.90,54,0,5.14,AZ,1724771801,
205,205,port alfred,-33.5906,26.8910,21.77,60,0,0.89,ZA,1724771814,
248,248,greenwood,34.1668,-82.1165,29.34,59,0,2.57,US,1724771864,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 100
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address =  requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
palmer - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
tolanaro - nearest hotel: Hôtel Mahavokey
majdal shams - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
bilajer - nearest hotel: Atlantis Otel \ Bilgəh
port alfred - nearest hotel: No hotel found
greenwood - nearest hotel: Hospitality Suites
colares - nearest hotel: Quinta de São Tadeu
vincennes - nearest hotel: Le Home
talara - nearest hotel: Hospedaje El Aypate
colonia - nearest hotel: Wasserturm Hotel Cologne
grand island - nearest hotel: Grand Island Travelodge
santa cruz - nearest hotel: Aviador Hotel Boutique
newman - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
plouguerneau - nearest hotel: An Aber Hôtel Café
eastham - nearest hotel: Four Points by Sheraton
toliara - nearest hotel: Ambary
ocean city - nearest hotel: Watson's Regency Suites
miramichi 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,palmer,42.1584,-72.3287,26.29,57,0,1.52,US,1724771573,No hotel found
16,16,crane,31.3974,-102.3501,25.33,60,0,2.68,US,1724771587,No hotel found
81,81,bethel,41.3712,-73.4140,27.17,67,0,0.89,US,1724771667,Hampton Inn Danbury
96,96,tolanaro,-25.0319,46.9987,22.07,79,0,8.51,MG,1724771684,Hôtel Mahavokey
120,120,majdal shams,33.2684,35.7694,22.37,41,0,4.05,IL,1724771712,No hotel found
166,166,tsiombe,-25.3000,45.4833,22.84,62,0,3.15,MG,1724771768,No hotel found
184,184,lata,40.1629,-8.3327,29.72,36,0,4.60,PT,1724771789,Residencial Botânico
194,194,bilajer,40.5681,50.0378,27.90,54,0,5.14,AZ,1724771801,Atlantis Otel \ Bilgəh
205,205,port alfred,-33.5906,26.8910,21.77,60,0,0.89,ZA,1724771814,No hotel found
248,248,greenwood,34.1668,-82.1165,29.34,59,0,2.57,US,1724771864,Hospitality Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "Hotel Name",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)

In [ ]:
### END ###